In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem

import parmed as pmd
import openmm
import openmm.app as app
import openmm.unit as u
import numpy as np

In [2]:
def compare_energies(psf_fn, positions, toppar_filenames, system_kwargs=None, units=u.kilojoules_per_mole):
    openmm_toppar = app.CharmmParameterSet(*toppar_filenames)
    openmm_psf = app.CharmmPsfFile(psf_fn)
    openmm_system = openmm_psf.createSystem(openmm_toppar, **system_kwargs)

    integrator = openmm.VerletIntegrator(1.0)
    platform = openmm.Platform.getPlatformByName("CPU")
    simulation = app.Simulation(openmm_psf.topology, openmm_system, integrator, platform)
    
    simulation.context.setPositions(positions)

    print(simulation.context.getState(getEnergy=True).getPotentialEnergy())
    simulation.minimizeEnergy(maxIterations=500)
    print(simulation.context.getState(getEnergy=True).getPotentialEnergy())
    optimized_positions = simulation.context.getState(getPositions=True)
    
    return optimized_positions.getPositions()

In [3]:
pdb_chignolin = pmd.load_file("1uao.pdb")
pdb_chignolin.visualize()

NGLWidget()

In [4]:
perturbed_locations = [x * np.random.uniform(0.9, 1.1) for x in pdb_chignolin.positions]
mm_perturbed_locations = openmm.unit.quantity.Quantity(perturbed_locations)
pdb_chignolin.positions = mm_perturbed_locations
pdb_chignolin.visualize()

NGLWidget()

In [49]:
SOLVENT_KWARGS = {
    "nonbondedMethod": app.NoCutoff,
    "constraints": app.HBonds,
    "implicitSolvent": app.HCT,
    "rigidWater": True,
}

chignolin_psf = "1uao_v2.psf"
optimized_positions = compare_energies(
    chignolin_psf,
    pdb_chignolin.positions,
    ["toppar/par_all36_prot.prm", "toppar/top_all36_prot.rtf","toppar/toppar_water_ions.str"], 
    system_kwargs=SOLVENT_KWARGS
)

OpenMMException: Called setPositions() on a Context with the wrong number of positions

In [6]:
pdb_chignolin.positions = optimized_positions
pdb_chignolin.visualize()

NGLWidget()

In [46]:
def generate_psf(pdb):
    mol = Chem.rdmolfiles.MolFromPDBFile(pdb, removeHs=False)
    mol = Chem.AddHs(mol)
    Chem.rdDistGeom.EmbedMolecule(mol)
    Chem.rdForceFieldHelpers.MMFFOptimizeMolecule(mol)
    
    intermediate_pdb = "temp.pdb"
    Chem.rdmolfiles.MolToPDBFile(mol, intermediate_pdb) # PDB with explicit Hydrogens
    return intermediate_pdb

In [48]:
chignolin_fn = generate_psf("1uao.pdb")
pdb_chignolin = pmd.load_file(pdb_fn)
pdb_chignolin.visualize()

NGLWidget()